In [1]:
import numpy as np
import os, sys
import warnings, functools
import mdtraj as md
import itertools

In [33]:
import importlib
def source_module(module_file: str, local_module_name: str = None):
    """to add a module from a user defined python script into the local name space"""
    if local_module_name is None:
        local_module_name = module_file.split("/")[-1].replace(".py", "")
    if len(module_file.split("/")) == 1 or module_file.split("/")[-2] == ".":
        module_dir = os.getcwd()
    else:
        module_dir = "/".join(module_file.split("/")[:-1])
    sys.path.insert(0, module_dir)
    module = importlib.import_module(module_file.split("/")[-1].replace(".py", ""))
    g = globals()
    g[local_module_name] = module
    pass
source_module("/Users/anjalidhar/Desktop/parsing_functions.py")

In [2]:
def num_str(s, return_str=True, return_num=True):
    
    s = ''.join(filter(str.isdigit, s))

    if return_str and return_num:
        return s, int(s)

    if return_str:
        return s

    if return_num:
        return int(s)
    
def sort_strs(strs: list, max=False, indexed: bool=False):
    
    """ strs ::: a list or numpy array of strings.
        max ::: bool, to sort in terms of greatest index to smallest.
        indexed ::: bool, whether or not to filter out strings that don't contain digits.
                    if set to False and string list (strs) contains strings without a digit, function 
                    will return unsorted string list (strs) as an alternative to throwing an error."""
    
    # we have to ensure that each str in strs contains a number otherwise we get an error
    check = np.vectorize(lambda s : any(map(str.isdigit, s)))
    
    if isinstance(strs, list):
        strs = np.array(strs)
    
    # the indexed option allows us to filter out strings that don't contain digits.
    ## This prevents an error
    if indexed:
        strs = strs[check(strs)]

    #if indexed != True, then we don't filter the list of input strings and simply return it
    ##because an attempt to sort on indices (digits) that aren't present results in an error
    else:
        if not all(check(strs)):
            
            warnings.warn("Not all strings contain a number, returning unsorted input list to avoid throwing an error. "
                        "If you want to only consider strings that contain a digit, set indexed to True ")
            
            return strs
    
    get_index = np.vectorize(functools.partial(num_str, return_str=False, return_num=True))
    indices = get_index(strs).argsort()
    
    if max:
        return strs[np.flip(indices)]
    
    else:
        return strs[indices]
def lsdir(dir, keyword: "list or str" = None,
          exclude: "list or str" = None,
          indexed:bool=False):
    
    """ full path version of os.listdir with files/directories in order
    
        dir ::: path to a directory (str), required
        keyword ::: filter out strings that DO NOT contain this/these substrings (list or str)=None
        exclude ::: filter out strings that DO contain this/these substrings (list or str)=None
        indexed ::: filter out strings that do not contain digits.
                    Is passed to sort_strs function (bool)=False"""

    if dir[-1] == "/":
        dir = dir[:-1] # slicing out the final '/'

    listed_dir = os.listdir(dir) # list out the directory 

    if keyword is not None:
        listed_dir = keyword_strs(listed_dir, keyword) # return all items with keyword
    
    if exclude is not None:
        listed_dir = keyword_strs(listed_dir, keyword=exclude, exclude=True) # return all items without excluded str/list

    # Sorting (if possible) and ignoring hidden files that begin with "." or "~$"
    return [f"{dir}/{i}" for i in sort_strs(listed_dir, indexed=indexed) if (not i.startswith(".")) and (not i.startswith("~$"))] 

def keyword_strs(strs: list, keyword: "list or str", exclude: bool = False):
    
    if isinstance(keyword, str): # if the keyword is just a string 
        
        if exclude:
            filt = lambda string: keyword not in string
        
        else:
            filt = lambda string: keyword in string

    else:
        if exclude:
            filt = lambda string: all(kw not in string for kw in keyword)

        else:
            filt = lambda string: all(kw in string for kw in keyword)

    return list(filter(filt, strs))

def keyword_strs2(strs: list,
                 keyword: "list or str",
                 exclude: bool=False,
                 match:callable=all):

    if isinstance(keyword, str):
        if exclude:
            filt = lambda string: keyword not in string
        
        else:
            filt = lambda string: keyword in string

    else:
        if exclude:
            filt = lambda string: match(kw not in string for kw in keyword)

        else:
            filt = lambda string: match(kw in string for kw in keyword)

    return list(filter(filt, strs))

def filter_strs(strs:list,
                keyword: "list or str" = None,
                exclude: "list or str" = None,
                match:callable = all):

    if keyword is not None:
        strs = keyword_strs2(strs, keyword=keyword, exclude=False, match=match)

    if exclude is not None:
        strs = keyword_strs2(strs, keyword=exclude, exclude=True, match=match)

    return strs

In [16]:
dir = '/Users/anjalidhar/Desktop/notebooks/23W/gnina-gpu/47onapo/c_16_0_0'
gnina_dir = f'{dir}/out'

lig_mol2 = f'{dir}/47.mol2'
lig_pdbqt = f'{dir}/47.pdbqt'

protein_pdb = f'{dir}/protein.pdb'
protein_pdbqt = f'{dir}/protein.pdbqt'

protein_traj = md.load(protein_pdb)

In [19]:
def return_scores_ligsdf_2(gnina_outdir): 
    """ For a given gnina frame out directory, will return the filepath of 
    the isolated lowest-docking score ligand sdf and the score. 
    Saves a numpy array of docking scores for all residues of the frame in the directory. 
    NOTE: picks up ligand files from the string 'ligout' ."""

    # Getting lowest score and saving all the scores to an array 
    ligout_files = lsdir(gnina_outdir, keyword='ligout')
    scores = []
    for ligfile in ligout_files: 
        with open(ligfile, 'r') as openfile: 
            lines = openfile.readlines()
            if len(lines)>0: # if we actually have something written in the file
                sc_lines = filter_strs(lines, keyword = 'minimizedAffinity', match=any)
                marker2 = lines.index(sc_lines[0])
                scores.append(float(lines[marker2+1]))
            else: 
                scores.append(0) # otherwise, just append a score of 0 to indicate this frame failed. 
    np.save(f'{gnina_outdir}/frame_score.npy', np.asarray(scores))

    ######

    # Now, parsing the lowest score residue for the ligand xyz
    best_ligout_file = ligout_files[scores.index(min(scores))]
    with open(best_ligout_file, 'r') as openfile: 
        lines = openfile.readlines()
        # getting the best docking score and first marker
        score_lines = filter_strs(lines, keyword = 'minimizedAffinity', match=any)
        marker3 = lines.index(score_lines[0])
        lowest_docking_score = float(lines[marker3+1])

        # find the index of the first 'openbabel' line: split with the score line to get the whole molecule. 
        babel_lines = filter_strs(lines, keyword = 'OpenBabel', match=any)
        marker1 = lines.index(babel_lines[0])
        ligand_lines = lines[marker1+2:marker3]

    with open(f"{gnina_outdir}/best_{121+scores.index(min(scores))}_out.sdf", 'w+') as written_file: 
        for line in ligand_lines: 
            written_file.write(line)
    
    # the, returning the sdf file of just the best docked ligand. 
    return f"{gnina_outdir}/best_{121+scores.index(min(scores))}_out.sdf", lowest_docking_score, scores.index(min(scores))
    

In [20]:
lig_file, score, best_idx = return_scores_ligsdf_2(gnina_dir)

In [7]:
# For generating flexible residue list! 

def intra_residue_distances(traj, d0:int=1): 
    """ Returns a distance matrix of shape (n_frames, n_res, n_res) of the intramolecular distances 
    between residues of a given trajectory object. """

    def combinations(x):
        return np.asarray(list(itertools.combinations(x, 2)))
    
    n = len([res for res in traj.topology.residues])
    index_0 = np.arange(n)
    indices = combinations(index_0)
    distances = md.compute_contacts(traj, indices)[0]*10

    # info about flattened distance matrix
    N, d = distances.shape

    matrix = np.zeros([N] + [n] * 2)
    i, j = np.triu_indices(n, d0)
    matrix[:, i, j] = distances
    return matrix + matrix.transpose(0, 2, 1)

def return_flex_residues(pro_traj_obj,  cutoff=5, flex_res_cut=None):
    """ Returns a matrix with shape (n_frames, n_res, n_res) or (n_frames, n_res, flex_res_cutoff)
    the resSeqs of residues within the cutoff to the docking residue of question.   
    If not, will return zeroes in place of resSeq."""
    residues = [res.resSeq for res in pro_traj_obj.topology.residues]
    # residue intramolecular distances --> shape: (n_frames, n_res, n_res)
    n_res = len(residues)
    intra_dist = intra_residue_distances(pro_traj_obj)

    if flex_res_cut==None: 
        res_idx = np.zeros((intra_dist.shape[0],n_res, n_res))
        for i, frame in enumerate(intra_dist):
            for j, res in enumerate(frame): 
                sorted_idx = np.argsort(res) # will always include the residue of question, so ignore it
                for k, idx in enumerate(sorted_idx):
                    if res[idx]<cutoff: 
                        res_idx[i][j][k] = residues[idx]
        return res_idx

    else: 
        #Initializing matrix for residue indices we choose 
        res_idx = np.zeros((int(intra_dist.shape[0]),int(intra_dist.shape[1]),int(flex_res_cutoff)))
        for i, frame in enumerate(intra_dist):
            for j, res in enumerate(frame): 
                sorted_idx = np.argsort(res)[:flex_res_cutoff] # including docking residue
                for k, idx in enumerate(sorted_idx):
                    if res[idx]<cutoff: 
                        res_idx[i][j][k] = residues[idx]
        return res_idx

In [8]:
# the index for the lowest energy binding mode for the one frame
best_idx

11

In [14]:
# a matrix with all of the residues made flexible for each docking run, on each residue 
flex_mat = return_flex_residues(protein_traj,  cutoff=5, flex_res_cut=None)
flex_mat.shape

(1, 20, 20)

In [10]:
# loading in the non flexible residues
frame = 0
not_flexible_residues = [124, 128, 132, 138, 140]

In [21]:
# now, pulling the flexible residues for the lowest energy docking residue 
# disregarding those that aren't flexible
flex_idx = [int(x) for x in list(flex_mat[frame][best_idx]) if x not in not_flexible_residues and x!=0]
flex_idx.sort()

In [207]:
keyword_strs(["CA"], keyword="A", exclude=True)

[]

In [188]:
# the residues, in order, that were made flexible for the lowest energy docking residue 
flex_idx 

[129, 130, 131, 133, 134]

In [31]:
# loading in
flex_protein_sdfs = lsdir(gnina_dir, keyword='protein_out')
pro_out_sdf = flex_protein_sdfs[best_idx]

In [176]:
for r in set(protein_traj.top.residues):
    print(list(r.atoms))


[MET127-N, MET127-CA, MET127-C, MET127-O, MET127-CB, MET127-CG, MET127-SD, MET127-CE, MET127-H, MET127-HA, MET127-HB2, MET127-HB3, MET127-HG2, MET127-HG3, MET127-HE1, MET127-HE2, MET127-HE3]
[ALA140-N, ALA140-CA, ALA140-C, ALA140-O, ALA140-CB, ALA140-OXT, ALA140-H, ALA140-HA, ALA140-HB1, ALA140-HB2, ALA140-HB3]
[TYR133-N, TYR133-CA, TYR133-C, TYR133-O, TYR133-CB, TYR133-CG, TYR133-CD1, TYR133-CD2, TYR133-CE1, TYR133-CE2, TYR133-CZ, TYR133-OH, TYR133-H, TYR133-HA, TYR133-HB2, TYR133-HB3, TYR133-HD1, TYR133-HD2, TYR133-HE1, TYR133-HE2, TYR133-HH]
[PRO128-N, PRO128-CA, PRO128-C, PRO128-O, PRO128-CB, PRO128-CG, PRO128-CD, PRO128-HA, PRO128-HB2, PRO128-HB3, PRO128-HG2, PRO128-HG3, PRO128-HD2, PRO128-HD3]
[ASP121-N, ASP121-CA, ASP121-C, ASP121-O, ASP121-CB, ASP121-CG, ASP121-OD1, ASP121-OD2, ASP121-H, ASP121-HA, ASP121-HB2, ASP121-HB3, ASP121-H, ASP121-H]
[TYR125-N, TYR125-CA, TYR125-C, TYR125-O, TYR125-CB, TYR125-CG, TYR125-CD1, TYR125-CD2, TYR125-CE1, TYR125-CE2, TYR125-CZ, TYR125-OH, TYR1

In [189]:
sidechain_traj = protein_traj.atom_slice(protein_traj.top.select("sidechain or name CA or name "))

In [217]:
flex_idx

[129, 130, 131, 133, 134]

In [216]:
sdf

<mdtraj.Trajectory with 1 frames, 39 atoms, 5 residues, without unitcells at 0x166080bd0>

In [215]:
hydrogen_dict = {
    "ALA":[""], 
    "VAL":[""], 
    "ILE":[""],
    "VAL":[""],
    "LEU":[""], 
    "PHE":[""], 
    "MET":[''],
    "PRO":[''],
    "GLY":[""],
    "ASP":[""],
    "GLU":[''],
    "TRP":[""], 
    "CYS":[''],
    "SER":['HG'], 
    "THR":[''],
    "ASN":['HD21', 'HD22'],
    "GLN":['HE21', 'HE22'],
    "ARG":[""],
    "HIS":[""],
    "LYS":[""],
    "TYR":["HH"]
    }

In [214]:
list(protein_traj.top.residues)

[ASP121,
 ASN122,
 GLU123,
 ALA124,
 TYR125,
 GLU126,
 MET127,
 PRO128,
 SER129,
 GLU130,
 GLU131,
 GLY132,
 TYR133,
 GLN134,
 ASP135,
 TYR136,
 GLU137,
 PRO138,
 GLU139,
 ALA140]

In [212]:
fxn = lambda x : ~any((i in x for i in ("A", "B")))
for r in sidechain_traj.top.residues:
   print(r)
   strs = list(map(str,list(r.atoms)))
   strs = [s.split("-")[-1] for s in strs]
   strs = filter_strs(strs, exclude="A,B".split(","), keyword="H", match=any)
   print(strs)



ASP121
['HB2', 'HB3']
ASN122
['HB2', 'HB3', 'HD21', 'HD22']
GLU123
['HB2', 'HB3', 'HG2', 'HG3']
ALA124
['HB1', 'HB2', 'HB3']
TYR125
['OH', 'HB2', 'HB3', 'HD1', 'HD2', 'HE1', 'HE2', 'HH']
GLU126
['HB2', 'HB3', 'HG2', 'HG3']
MET127
['HB2', 'HB3', 'HG2', 'HG3', 'HE1', 'HE2', 'HE3']
PRO128
['HB2', 'HB3', 'HG2', 'HG3', 'HD2', 'HD3']
SER129
['HB2', 'HB3', 'HG']
GLU130
['HB2', 'HB3', 'HG2', 'HG3']
GLU131
['HB2', 'HB3', 'HG2', 'HG3']
GLY132
['HA2', 'HA3']
TYR133
['OH', 'HB2', 'HB3', 'HD1', 'HD2', 'HE1', 'HE2', 'HH']
GLN134
['HB2', 'HB3', 'HG2', 'HG3', 'HE21', 'HE22']
ASP135
['HB2', 'HB3']
TYR136
['OH', 'HB2', 'HB3', 'HD1', 'HD2', 'HE1', 'HE2', 'HH']
GLU137
['HB2', 'HB3', 'HG2', 'HG3']
PRO138
['HB2', 'HB3', 'HG2', 'HG3', 'HD2', 'HD3']
GLU139
['HB2', 'HB3', 'HG2', 'HG3']
ALA140
['HB1', 'HB2', 'HB3']


In [188]:
list(protein_traj.top.residue(0).atoms)

[ASP121-N,
 ASP121-CA,
 ASP121-C,
 ASP121-O,
 ASP121-CB,
 ASP121-CG,
 ASP121-OD1,
 ASP121-OD2,
 ASP121-H,
 ASP121-HA,
 ASP121-HB2,
 ASP121-HB3,
 ASP121-H,
 ASP121-H]

In [36]:
flex_xyz = parsing_functions.ligand_xyz(pro_out_sdf)

In [59]:
sele_str = lambda l : " or ".join([f"residue {i}" for i in l])
sele_str(flex_idx)

'residue 129 or residue 130 or residue 131 or residue 133 or residue 134'

In [61]:
flex_traj = protein_traj.atom_slice(protein_traj.top.select(sele_str(flex_idx)))
true_top = flex_traj.top

In [ ]:
def resort_traj(traj, indices): 
    """ Resorts a trajectory from a set of indices. """ 
    table, bonds = traj.topology.to_dataframe()
    # Sort based on indices, then reset the table index 
    table = table.reindex(indices).set_index(np.arange(0,len(table)))
    # replace the old atom index to be ascending i.e 1,2,3....N 
    table["serial"] = np.arange(1,len(table)+1)
    # The new topology from the data frame 
    top = md.Topology.from_dataframe(table,bonds)
    # And then the loading the new traj from the new top! 
    newtraj = md.Trajectory(xyz = traj.xyz[:,indices], topology=top)
    return newtraj
    

In [108]:
from openbabel import openbabel
def sdf_to_pdb(sdf, pdb):
    """ Converts sdf files to pdb files using open babel. """ 
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("sdf", "pdb")
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, sdf) 
    obConversion.WriteFile(mol, pdb) 
    return pdb

In [107]:
sdf_pdb = sdf_to_pdb(pro_out_sdf, '/Users/anjalidhar/Desktop/sdf_pdb.pdb')

/Users/anjalidhar/anaconda3/envs/test/lib/python3.11/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


In [184]:
len(hydrogen_dict.keys())

20

In [ ]:
def replace_sidechain(traj0, traj1):
    



In [ ]:
"(residue 129) and ((sidechain and (type H bonded name O)) or type C or (sidechain and not type H))"
"(residue 129) and ((sidechain) or type C or (type H bonded name O))"
"(residue 129) and (sidechain and (type H bonded name O) )"

In [219]:
a = 9
a = a*10 if a == 9 else 0

In [229]:
hydrogen_dict

{'ALA': [''],
 'VAL': [''],
 'ILE': [''],
 'LEU': [''],
 'PHE': [''],
 'MET': [''],
 'PRO': [''],
 'GLY': [''],
 'ASP': [''],
 'GLU': [''],
 'TRP': [''],
 'CYS': [''],
 'SER': ['HG'],
 'THR': [''],
 'ASN': ['HD21', 'HD22'],
 'GLN': ['HE21', 'HE22'],
 'ARG': [''],
 'HIS': [''],
 'LYS': [''],
 'TYR': ['HH']}

In [235]:
def fmtr(index:int, residue:str):
    sel = (" or name "+" or name ".join(hydrogen_dict[residue]) if len(hydrogen_dict[residue])>1 else f" or name {hydrogen_dict[residue][0]}"
          if hydrogen_dict[residue] != "" else "")

    return f"residue {index} and not type H and (sidechain or name CA or name C){sel}"

In [237]:
protein_traj.atom_slice(protein_traj.top.select(fmtr(129, "ASN")))

<mdtraj.Trajectory with 1 frames, 6 atoms, 2 residues, without unitcells at 0x1136dfc50>

In [236]:
fmtr(129, "ASN")

'residue 129 and not type H and (sidechain or name CA or name C) or name HD21 or name HD22'

In [238]:
real_1 = protein_traj.atom_slice(protein_traj.top.select(fmtr(129, "SER")))
sdf_1 = sdf.atom_slice(sdf.top.select("residue 1"))

print(list(real_1.top.atoms))
print(list(sdf_1.top.atoms))

print(real_1.xyz)
print(sdf_1.xyz)


[SER129-CA, SER129-C, SER129-CB, SER129-OG, SER129-HG]
[UNL1-C, UNL1-C, UNL1-C, UNL1-O, UNL1-H]
[[[ 0.888 -0.106 -0.24 ]
  [ 0.754 -0.076 -0.307]
  [ 0.934 -0.249 -0.291]
  [ 0.86  -0.347 -0.218]
  [ 0.893 -0.435 -0.236]]]
[[[ 0.888  -0.106  -0.24  ]
  [ 0.754  -0.076  -0.307 ]
  [ 0.934  -0.249  -0.291 ]
  [ 0.818  -0.3321 -0.2977]
  [ 0.8421 -0.4229 -0.3158]]]


In [110]:
sdf = md.load(sdf_pdb)

In [103]:
atoms = [f"{i}{j}" for j,i in enumerate(atoms)]

In [97]:
with open("/Users/anjalidhar/scratch", "r") as file:
    atoms = file.readlines()

In [100]:
import os
atoms = [i.replace(os.linesep, "") for i in atoms[1:]]

In [84]:
md.Topology(pd.DaataFrame(flex_xyz[:79]))

TypeError: Topology.__init__() takes 1 positional argument but 2 were given

In [66]:
fix_traj = md.Trajectory(flex_xyz[:79], topology=true_top)

In [80]:
fix_traj.atom_slice(fix_traj.top.select("resid 0")).xyz

array([[[ 8.88 , -1.06 , -2.4  ],
        [ 7.54 , -0.76 , -3.07 ],
        [ 9.34 , -2.49 , -2.91 ],
        [ 8.18 , -3.321, -2.977],
        [ 8.421, -4.229, -3.158],
        [ 6.46 , -0.14 , -5.3  ],
        [ 5.89 , -1.51 , -5.66 ],
        [ 6.94 ,  0.63 , -6.55 ],
        [ 6.741,  2.134, -6.548],
        [ 8.098,  2.767, -6.325],
        [ 8.477,  3.73 , -7.085]]], dtype=float32)

In [113]:
351/79

4.443037974683544

In [112]:
flex_xyz.shape

(351, 3)

In [73]:
list(fix_traj.top.residue(0).atoms)


[SER129-N,
 SER129-CA,
 SER129-C,
 SER129-O,
 SER129-CB,
 SER129-OG,
 SER129-H,
 SER129-HA,
 SER129-HB2,
 SER129-HB3,
 SER129-HG]

In [69]:
list(fix_traj.top.residues)

[SER129, GLU130, GLU131, TYR133, GLN134]

In [29]:
# parsing? 

# only reading in the frame with the lowest docking energy
with open(flex_protein_sdfs[best_idx], 'r') as openfile: 
    lines = openfile.readlines()
    n_atoms = int(lines[3].split(' ')[1])
    xyz_lines = lines[3:4+n_atoms]

for line in xyz_lines:
    print(line)
    


 39 35  0  0  0  0  0  0  0  0999 V2000

    8.8800   -1.0600   -2.4000 C   0  0  0  0  0  2  0  0  0  0  0  0

    7.5400   -0.7600   -3.0700 C   0  0  0  0  0  1  0  0  0  0  0  0

    9.3400   -2.4900   -2.9100 C   0  0  0  0  0  2  0  0  0  0  0  0

    8.1800   -3.3210   -2.9770 O   0  0  0  0  0  0  0  0  0  0  0  0

    8.4210   -4.2290   -3.1580 H   0  0  0  0  0  0  0  0  0  0  0  0

    6.4600   -0.1400   -5.3000 C   0  0  0  0  0  2  0  0  0  0  0  0

    5.8900   -1.5100   -5.6600 C   0  0  0  0  0  1  0  0  0  0  0  0

    6.9400    0.6300   -6.5500 C   0  0  0  0  0  2  0  0  0  0  0  0

    6.7410    2.1340   -6.5480 C   0  0  0  0  0  2  0  0  0  0  0  0

    8.0980    2.7670   -6.3250 C   0  0  0  0  0  0  0  0  0  0  0  0

    8.4770    3.7300   -7.0850 O   0  0  0  0  0  0  0  0  0  0  0  0

    8.7320    2.4400   -5.2860 O   0  0  0  0  0  1  0  0  0  0  0  0

    6.3900   -4.0000   -5.8800 C   0  0  0  0  0  2  0  0  0  0  0  0

    7.6200   -4.8200   -5.5400 C   0